### KoBART 를 이용한 텍스트 요약 시스템 구성
#### author: judepark@jbnu.ac.kr

- 이미 학습되어있는 모델을 불러와서 텍스트 요약 시스템을 돌려봅니다.

In [1]:
from transformers import BartModel, PreTrainedTokenizerFast, BartForConditionalGeneration
import torch

In [2]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

text = "과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다."
print(f'입력 문서: {text}')

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]))
print(f'요약 결과: {tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)}')

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/111 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

입력 문서: 과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다.
요약 결과: TV가 없는 집도 많아지고 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다.


### 텍스트 요약 시스템 학습 (Fine-tuning) 하기
### 1. 데이터 확인하기

In [5]:
with open('./data/train.tsv', 'r', encoding='utf-8') as f:
    train_dataset = [i.strip() for i in f][1:101]

with open('./data/test.tsv', 'r', encoding='utf-8') as f:
    test_dataset = [i.strip() for i in f][1:11]

In [6]:
len(train_dataset), len(test_dataset)

(100, 10)

In [7]:
train_dataset[:1]

["일제 강제징용피해자들이 지난 2월 발의된 ‘근로정신대 피해자 지원법률안’의 조속한 통과를 촉구 했다. 시민단체 ‘근로정신대 할머니와 함께 하는 시민모임’은 13일 광주시의회 시민 소통실에서 기자회견을 열고 “피해자들을 더 이상 외면하지 말고 지원법을 제정해달라”고 주장했다. 시민모임은 “광복 74년이 됐지만, 여자근로정신대 피해자들은 아직도 일제가 씌운 굴레에서 벗어나지 못하고 있는 처지”라면서 일본정부와 한국정부 양쪽 모두의 잘못을 지적했다. 이들은 “해방 후 진실규명과 명예회복을 전혀 하지 않은 일본정부의 잘못이 가장 크다”고 비판했다. 이어 “우리나라의 많은 국민이 ‘근로정신대’와 일본군 ‘위안부’ 문제를 아예 구별하지 못하고 있는 것은 제대로 된 진상조사나 역사교육을 하지 않고 있는 우리 정부의 탓도 있다”고 덧붙였다. 이들은 또 “어렵게 용기를 내 근로정신대 피해자라고 고백을 하더라도 위안부 피해자와 달리 국가로부터 어떤 지원이나 위로도 받을 수 없었다”며 “이런 현실 앞에 한국 정부는 그 책임에서 자유로울 수 없다”고 설명했다. 시민모임은 “이미 지난 2월 바른미래당 김동철(광주 광산갑) 의원이 근로정신대 피해자 지원법률안을 발의했지만, 국회에선 담당 상임위원회에 상정조차 안되고 있다”면서 “근로정신대 피해자들을 위해 조속히 법률 제정에 나서야 한다”고 요구했다.\t13일 광주시의회 시민 소통실에서 시민단체 '근로정신대 할머니와 함께 하는 시민모임'은 기자회견을 열어 지난 2월 발의된 '근로정신대 피해자 지원법률안'의 조속한 통과를 요구했다."]

데이터는 입력 문서/요약문으로 구성되어 있습니다. 여기서 \t로 입력 문서/요약문을 구분합니다.

### 2. PyTorch Dataset 정의

In [8]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from torch.utils.data import Dataset, DataLoader, IterableDataset

class KoBARTSummaryDataset(Dataset):
    def __init__(self, file, tok, max_len, pad_index = None, ignore_index=-100):
        super().__init__()
        self.tok = tok
        self.max_len = max_len
        self.docs = pd.read_csv(file, sep='\t')
        self.len = self.docs.shape[0]
        if pad_index is None:
            self.pad_index = self.tok.pad_token_id
        else:
            self.pad_index = pad_index
        self.ignore_index = ignore_index

    def add_padding_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs

    def add_ignored_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.ignore_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs
    
    def __getitem__(self, idx):
        instance = self.docs.iloc[idx]
        input_ids = self.tok.encode(instance['news'])
        input_ids = self.add_padding_data(input_ids)

        label_ids = self.tok.encode(instance['summary'])
        label_ids.append(self.tok.eos_token_id)
        dec_input_ids = [self.pad_index]
        dec_input_ids += label_ids[:-1]
        dec_input_ids = self.add_padding_data(dec_input_ids)
        label_ids = self.add_ignored_data(label_ids)

        return {'input_ids': np.array(input_ids, dtype=np.int_),
                'decoder_input_ids': np.array(dec_input_ids, dtype=np.int_),
                'labels': np.array(label_ids, dtype=np.int_)}
    
    def __len__(self):
        return self.len

In [9]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("hyunwoongko/kobart")
model = BartForConditionalGeneration.from_pretrained("hyunwoongko/kobart")

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

In [10]:
train_sum_dataset = KoBARTSummaryDataset('./data/train.tsv', tokenizer, max_len=128)
test_sum_dataset = KoBARTSummaryDataset('./data/test.tsv', tokenizer, max_len=128)

In [11]:
train_dataloader = DataLoader(train_sum_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(train_sum_dataset, batch_size=8, shuffle=False)

### 2. 모델 학습하기

In [12]:
from torch.optim import Adam
from tqdm import tqdm

In [13]:
parameters = model.parameters()
optimizer = Adam(parameters, lr=5e-5)

In [15]:
epoch = 1 # 1 epoch 만 학습합니다.
is_cuda = True # GPU 환경입니다. 

if is_cuda:
    model.cuda()

for e in range(epoch):
    pbar = tqdm(train_dataloader, total=len(train_dataloader))
    for batch in pbar:
        optimizer.zero_grad()
        batch = {k: v.cuda() for k, v in batch.items()} if is_cuda else batch
        output = model(**batch)
        loss = output.loss
        
        loss.backward()
        optimizer.step()
        pbar.set_description(f'epoch: {e} loss: {loss}')

epoch: 0 loss: 2.5689656734466553: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 4281/4281 [08:17<00:00,  8.60it/s]


In [17]:
text = "과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다."
print(f'입력 문서: {text}')

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]).cuda())
print(f'요약 결과: {tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)}')

입력 문서: 과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다.
요약 결과: TV가 보는 재미와 함께 TV를 보는 재미도 함께 느낄 수 있다.
